In [1]:
import constti
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import numpy as np
from pathlib import Path

team_number = 20
url1 = "https://fantasy.premierleague.com/api/bootstrap-static/"
url2 = "https://fantasy.premierleague.com/api/entry/698498/history/"
url3 = "https://fantasy.premierleague.com/api/event/6/live/"
url4 = "https://fantasy.premierleague.com/api/fixtures"

#NaNs to zeros
def toint(a):
    if np.isnan(a):
        return 0
    else: return int(a)

#If no matches played not to devide by zero
def noZ(a):
    b = a.copy()
    for i in range(len(b)):
        if b[i] == 0:
            b[i]=1
    return b

#Kills unfinished matches
def is_finished(n):
    if n=='':
        return False
    else:
        a = Fixtures[Fixtures['id']==n]['finished']
        #print(a)
        return a.bool()

p1 = requests.get(url1)
#page1 = BeautifulSoup(p1.text)
#data1 = str(page1.p)[3:-4]
data1 = p1.text

d1 = json.loads(data1)
bigTable = pd.DataFrame(d1['elements'])
bigTable = bigTable[['team', 'element_type', 'web_name', 'goals_scored', 'assists', 'bonus', 'event_points', 'total_points', 
               'saves', 'own_goals', 'clean_sheets', 'penalties_missed', 'penalties_saved', 'yellow_cards', 'red_cards', 
               'minutes', 'bps', 'creativity', 'threat', 'ict_index', 'influence',
               'value_season', 'form', 'value_form', 'points_per_game', 
               'goals_conceded', 
               'in_dreamteam', 'dreamteam_count',
               'now_cost', 'cost_change_event', 'cost_change_event_fall',
               'cost_change_start', 'cost_change_start_fall', 'selected_by_percent',
               'transfers_in_event', 'transfers_out_event', 'transfers_in', 'transfers_out', 
               'chance_of_playing_this_round', 'chance_of_playing_next_round', 'news_added', 'news', 'status', 
               'ep_this', 'ep_next', 'first_name', 'second_name', 'team_code', 'id', 'photo', 'special', 'squad_number', 'code']]
bigTable['full_name'] = bigTable['first_name'] + ' ' + bigTable['second_name']
bigTable.to_csv(Path('in/fpltable.csv'))

p4 = requests.get(url4)
#page4 = BeautifulSoup(p4.text)
#data4 = str(page4.p)[3:-4]
d4 = json.loads(p4.text)
Fixtures = pd.DataFrame(d4)
Fixtures.to_csv(Path('in/fplfixtures.csv'))

#Figuring out gameweek of the last ended match - lastGW

#if Fixtures.at[1,'minutes'] != 90:
#    lastGW = 0
#elif Fixtures.at[len(Fixtures)-1,'minutes'] == 90:
#    lastGW = Fixtures.at[len(Fixtures)-1,'event']
#else:
#    for i in range(1,len(Fixtures)):
#        if Fixtures.at[i,'minutes'] != 90:
#            lastGW = Fixtures.at[i-1,'event']
#            break
#lastGW = int(lastGW)

firstr = len(Fixtures)+1
lastr = 0
for i in range(len(Fixtures)):
    if Fixtures.at[i,'finished']==True:
        firstr = min(firstr, i)
        lastr = i
if firstr < len(Fixtures)+1:
    lastGW = int(Fixtures.at[lastr-1,'event'])
else: lastGW = 0
if lastr == len(Fixtures):
    lastGW = int(Fixtures.at[lastr,'event'])







Gameweeks = pd.DataFrame()
for i in range(1,2*team_number - 1):
    url = "https://fantasy.premierleague.com/api/event/" + str(i) + "/live/"
    p = requests.get(url)
    #page = BeautifulSoup(p.text)
    #data = str(page.p)[3:-4]
    d = json.loads(p.text)
    nexTour = pd.DataFrame(d['elements'])
    
    if not nexTour.empty:
        nt1 = pd.DataFrame(nexTour['stats'].tolist())
        nt1['id'] = nexTour['id']
        nt1['gameweek'] = i
        nt1['fixture'] = [nexTour.at[i,'explain'][0]['fixture'] if not nexTour.at[i,'explain']==[] \
                          else '' for i in nexTour.index]
        #nt1.index = nt1['gameweek']*1000+nt1['id']
        Gameweeks = Gameweeks.append(nt1, ignore_index=True)
        print(i)

 

teams = dict(zip(pd.DataFrame(d1['teams'])['id'],pd.DataFrame(d1['teams'])['name']))
players = dict(zip(bigTable['id'],bigTable['full_name']))
teamplayers = dict(zip(bigTable['id'],bigTable['team']))

Gameweeks['team'] = [teamplayers[i] for i in Gameweeks['id']]

Gameweeks['threat'] = pd.to_numeric(Gameweeks['threat'])
Gameweeks['creativity'] = pd.to_numeric(Gameweeks['creativity'])

Gameweeks['team_a'] = [Fixtures[(Fixtures['event'] == Gameweeks.at[i,'gameweek']) & \
                                         ((Fixtures['team_a'] == Gameweeks.at[i,'team'])|\
                                          (Fixtures['team_h'] == Gameweeks.at[i,'team']))]['team_a'] \
                            for i in Gameweeks.index]


#Kill postponed matches lines

#li =[]
#for i in Gameweeks.index:
#    if Gameweeks.at[i,'team_a'].empty:
#        li.append(i)
#Gameweeks = Gameweeks.drop(li)

li =[]
for i in Gameweeks.index:
    if not is_finished(Gameweeks.at[i,'fixture']):
        li.append(i)
Gameweeks = Gameweeks.drop(li)
        

Gameweeks['team_a'] = [int(Gameweeks.at[i,'team_a']) for i in Gameweeks.index]

Gameweeks['team_h'] = [int(Fixtures[(Fixtures['event'] == Gameweeks.at[i,'gameweek']) & \
                                         ((Fixtures['team_a'] == Gameweeks.at[i,'team'])|\
                                          (Fixtures['team_h'] == Gameweeks.at[i,'team']))]['team_h']) \
                            for i in Gameweeks.index]

Gameweeks['teamAgainst'] = [Gameweeks.at[i,'team_a'] if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                            else Gameweeks.at[i,'team_h'] \
                           for i in Gameweeks.index]
Gameweeks['side'] = ['home' if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                            else 'away' \
                           for i in Gameweeks.index]

del Gameweeks['team_a']
del Gameweeks['team_h']


Gameweeks.to_csv(Path('in/fplgameweeks.csv'))
Gameweeks

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,...,threat,ict_index,total_points,in_dreamteam,id,gameweek,fixture,team,teamAgainst,side
0,90,0,0,1,0,0,0,0,0,0,...,0.0,1.8,7,False,14,1,10,1,13,away
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,27,1,6,2,17,away
2,90,0,0,1,0,0,0,0,0,0,...,0.0,2.5,9,False,47,1,7,4,18,away
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,48,1,7,4,18,away
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,72,1,2,3,15,home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11677,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,559,21,209,20,18,away
11678,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,561,21,205,8,11,away
11679,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,570,21,205,8,11,away
11680,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,False,581,21,207,7,14,away


In [2]:
#Fixtures
Team_fixtures = pd.DataFrame()
for j in range(lastGW,0,-1): 
    Team_fixtures['GW'+str(j)] = [Fixtures[((Fixtures['team_a']==i)|(Fixtures['team_h']==i))&\
                                (Fixtures['event']==j)]['id'].values for i in range(1, team_number+1)]

Player_fixtures = pd.DataFrame()
for j in range(lastGW,0,-1): 
    Player_fixtures['GW'+str(j)] = [Gameweeks[(Gameweeks['id']==i)&\
                                (Gameweeks['gameweek']==j)]['fixture'].values for i in bigTable['id']]

#Matches
TeamMatches = pd.DataFrame()
TeamMatches['Matches'] = [len(Fixtures[Fixtures['finished']&((Fixtures['team_a']==i)|(Fixtures['team_h']==i))]) \
                              for i in range(1,team_number+1)]

PlayerMatches = pd.DataFrame()
PlayerMatches['id'] = bigTable['id']
PlayerMatches['Team number'] = [bigTable[bigTable['id'] == i]['team'].sum() for i in players.keys()]
PlayerMatches['Team'] = [teams[PlayerMatches.at[i,'Team number']] for i in range(len(players))]
PlayerMatches['Team games'] = [TeamMatches.at[PlayerMatches.at[i,'Team number']-1,'Matches'] for i in PlayerMatches.index]
PlayerMatches['Played'] = [len(Gameweeks[(Gameweeks['id']==i)&(Gameweeks['minutes']>0)]) \
                        for i in PlayerMatches['id']]


#Team tables

print(0)
#1. Creating  a table with average threat and GW threats for teams

TeamThreat = pd.DataFrame()
TeamThreat['id'] = pd.DataFrame(d1['teams'])['id']
TeamThreat['Team'] = pd.DataFrame(d1['teams'])['name']
TeamThreat['Threat av'] = np.zeros(len(TeamThreat))
TeamThreat['Matches'] = TeamMatches['Matches']

for j in range(lastGW,0,-1):
    TeamThreat['Threat GW'+str(j)] = [[] for _ in range(team_number)]
    for i in range(team_number):
        for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
            TeamThreat.at[i,'Threat GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==Team_fixtures.at[i, 'GW'+str(j)][k])&\
                                                           (Gameweeks['team']==i+1)]['threat'].sum())
    

TeamThreat['Threat av'] = [Gameweeks[Gameweeks['team']==i]['threat'].sum() for i in range(1,team_number+1)] \
    /noZ(TeamMatches['Matches'])

print(1)
#2. Creating  a table with average creativity and GW creativities for teams

TeamCreativity = pd.DataFrame()
TeamCreativity['id'] = pd.DataFrame(d1['teams'])['id']
TeamCreativity['Team'] = pd.DataFrame(d1['teams'])['name']
TeamCreativity['Creativity av'] = np.zeros(len(TeamCreativity))
TeamCreativity['Matches'] = TeamMatches['Matches']

for j in range(lastGW,0,-1):
    TeamCreativity['Creativity GW'+str(j)] = [[] for _ in range(team_number)]
    for i in range(team_number):
        for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
            TeamCreativity.at[i,'Creativity GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==Team_fixtures.at[i, 'GW'+str(j)][k])&\
                                                           (Gameweeks['team']==i+1)]['creativity'].sum())
    

TeamCreativity['Creativity av'] = [Gameweeks[Gameweeks['team']==i]['creativity'].sum() for i in range(1,team_number+1)] \
    /noZ(TeamMatches['Matches'])
print(2)
#3. Creating  a table with average threat allowed by teams and GW threat allowed

TableDefence = pd.DataFrame()
TableDefence['id'] = pd.DataFrame(d1['teams'])['id']
TableDefence['Team'] = pd.DataFrame(d1['teams'])['name']
TableDefence['Threat allowed av'] = np.zeros(len(TableDefence))
TableDefence['Matches'] = TeamMatches['Matches']

for j in range(lastGW,0,-1):
    TableDefence['Threat allowed GW'+str(j)] = [[] for _ in range(team_number)]
    for i in range(team_number):
        for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
            TableDefence.at[i,'Threat allowed GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']== \
                Team_fixtures.at[i, 'GW'+str(j)][k])&(Gameweeks['teamAgainst']==i+1)]['threat'].sum())
    

TableDefence['Threat allowed av'] = [Gameweeks[Gameweeks['teamAgainst']==i]['threat'].sum() for i in range(1,team_number+1)] \
        /noZ(TeamMatches['Matches'])

threatAllowedAv = TableDefence['Threat allowed av'].mean()
print(3)
#4. Creating  a table with average adjusted threat and GW threats adj for teams

TeamThreatAd = pd.DataFrame()
TeamThreatAd['id'] = pd.DataFrame(d1['teams'])['id']
TeamThreatAd['Team'] = pd.DataFrame(d1['teams'])['name']
TeamThreatAd['Threat av adj'] = np.zeros(len(TeamThreatAd))
TeamThreatAd['Matches'] = TeamMatches['Matches']

for j in range(lastGW,0,-1):
    TeamThreatAd['Threat GW'+str(j)+' adj'] = [[] for _ in range(team_number)]
    for i in range(team_number):
        for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
            if toint(Fixtures[Fixtures['id']==Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==i+1:
                TeamThreatAd.at[i,'Threat GW'+str(j)+' adj'].append(TeamThreat.at[i,'Threat GW'+str(j)][k]*threatAllowedAv/ \
                    TableDefence.at[toint(Fixtures[Fixtures['id']==\
                    Team_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
            else:
                  TeamThreatAd.at[i,'Threat GW'+str(j)+' adj'].append(TeamThreat.at[i,'Threat GW'+str(j)][k]*threatAllowedAv/\
                    TableDefence.at[toint(Fixtures[Fixtures['id']==\
                    Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])
            
            TeamThreatAd.at[i,'Threat av adj'] = TeamThreatAd.at[i,'Threat av adj']  + \
                TeamThreatAd.at[i,'Threat GW'+str(j)+' adj'][k]
    

TeamThreatAd['Threat av adj'] = TeamThreatAd['Threat av adj']/noZ(TeamMatches['Matches'])
print(4)
#5. Creating  a table with average adjusted creativity and GW creativities adj for teams

TeamCreativityAd = pd.DataFrame()
TeamCreativityAd['id'] = pd.DataFrame(d1['teams'])['id']
TeamCreativityAd['Team'] = pd.DataFrame(d1['teams'])['name']
TeamCreativityAd['Creativity av adj'] = np.zeros(len(TeamCreativityAd))
TeamCreativityAd['Matches'] = TeamMatches['Matches']

for j in range(lastGW,0,-1):
    TeamCreativityAd['Creativity GW'+str(j)+' adj'] = [[] for _ in range(team_number)]
    for i in range(team_number):
        for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
            if toint(Fixtures[Fixtures['id']==Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==i+1:
                TeamCreativityAd.at[i,'Creativity GW'+str(j)+' adj'].append(TeamCreativity.at[i,'Creativity GW'+str(j)][k]*\
                    threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                    Team_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
            else:
                  TeamCreativityAd.at[i,'Creativity GW'+str(j)+' adj'].append(TeamCreativity.at[i,'Creativity GW'+str(j)][k]*\
                    threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                    Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])
            
            TeamCreativityAd.at[i,'Creativity av adj'] = TeamCreativityAd.at[i,'Creativity av adj']  + \
                TeamCreativityAd.at[i,'Creativity GW'+str(j)+' adj'][k]
    

TeamCreativityAd['Creativity av adj'] = TeamCreativityAd['Creativity av adj']/noZ(TeamMatches['Matches'])
print(5)
#6. Creating  a table with average threat allowed adjusted by teams and GW threat allowed adjusted

TableDefenceAd = pd.DataFrame()
TableDefenceAd['id'] = pd.DataFrame(d1['teams'])['id']
TableDefenceAd['Team'] = pd.DataFrame(d1['teams'])['name']
TableDefenceAd['Threat allowed av adj'] = np.zeros(len(TableDefenceAd))
TableDefenceAd['Matches'] = TeamMatches['Matches']

for j in range(lastGW,0,-1):
    TableDefenceAd['Threat allowed GW'+str(j)+' adj'] = [[] for _ in range(team_number)]
    for i in range(team_number):
        for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
            if toint(Fixtures[Fixtures['id']==Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==i+1:
                TableDefenceAd.at[i,'Threat allowed GW'+str(j)+' adj'].append(TableDefence.at[i,'Threat allowed GW'+str(j)][k]*\
                    threatAllowedAv/TeamThreat.at[toint(Fixtures[Fixtures['id']==\
                    Team_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat av'])
            else:
                TableDefenceAd.at[i,'Threat allowed GW'+str(j)+' adj'].append(TableDefence.at[i,'Threat allowed GW'+str(j)][k]*\
                    threatAllowedAv/TeamThreat.at[toint(Fixtures[Fixtures['id']==\
                    Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat av'])
            
            TableDefenceAd.at[i,'Threat allowed av adj'] = TableDefenceAd.at[i,'Threat allowed av adj']  + \
                TableDefenceAd.at[i,'Threat allowed GW'+str(j)+' adj'][k]
    

TableDefenceAd['Threat allowed av adj'] = TableDefenceAd['Threat allowed av adj']/noZ(TeamMatches['Matches'])

print(6)


#Total Team Table

TableTeams = pd.DataFrame()
TableTeams['id'] = pd.DataFrame(d1['teams'])['id']
TableTeams['Team'] = pd.DataFrame(d1['teams'])['name']

TableTeams['Threat adjusted'] = TeamThreatAd['Threat av adj']
TableTeams['Threat'] = TeamThreat['Threat av']
TableTeams['Creativity adjusted'] = TeamCreativityAd['Creativity av adj']
TableTeams['Creativity'] = TeamCreativity['Creativity av']
TableTeams['Threat allowed adjusted'] = TableDefenceAd['Threat allowed av adj']
TableTeams['Threat allowed'] = TableDefence['Threat allowed av']
print(7)



#PLayer Tables


#1 Players Threat

PlayerThreat = pd.DataFrame()
PlayerThreat['id'] = bigTable['id']
PlayerThreat['Name'] = bigTable['full_name']
PlayerThreat['Team'] = PlayerMatches['Team']
PlayerThreat['Threat per fixture'] = np.zeros(len(players))
PlayerThreat['Threat per game'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerThreat['Threat GW'+str(j)] = [[] for _ in range(len(bigTable))]
    for i in range(len(bigTable)):
        for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):
            PlayerThreat.at[i,'Threat GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==Player_fixtures.at[i, 'GW'+str(j)][k])&\
                                                           (Gameweeks['id']==PlayerThreat.at[i,'id'])]['threat'].sum())
            PlayerThreat.at[i,'Threat per game'] = PlayerThreat.at[i,'Threat per game'] +\
                PlayerThreat.at[i,'Threat GW'+str(j)][k]
    

PlayerThreat['Threat per fixture'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Team games'])
PlayerThreat['Threat per game'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Played'])
print(9)
#2 Players Creativity

PlayerCreativity = pd.DataFrame()
PlayerCreativity['id'] = bigTable['id']
PlayerCreativity['Name'] = bigTable['full_name']
PlayerCreativity['Team'] = PlayerMatches['Team']
PlayerCreativity['Creativity per fixture'] = np.zeros(len(players))
PlayerCreativity['Creativity per game'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerCreativity['Creativity GW'+str(j)] = [[] for _ in range(len(bigTable))]
    for i in range(len(bigTable)):
        for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):
            PlayerCreativity.at[i,'Creativity GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==\
                                        Player_fixtures.at[i, 'GW'+str(j)][k])&\
                                        (Gameweeks['id']==PlayerCreativity.at[i,'id'])]['creativity'].sum())
            PlayerCreativity.at[i,'Creativity per game'] = PlayerCreativity.at[i,'Creativity per game'] +\
                PlayerCreativity.at[i,'Creativity GW'+str(j)][k]
    

PlayerCreativity['Creativity per fixture'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Team games'])
PlayerCreativity['Creativity per game'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Played'])
print(10)
#3 Players Threat Adjusted

PlayerThreatAd = pd.DataFrame()
PlayerThreatAd['id'] = bigTable['id']
PlayerThreatAd['Name'] = bigTable['full_name']
PlayerThreatAd['Team number'] = PlayerMatches['Team number']
PlayerThreatAd['Team'] = PlayerMatches['Team']
PlayerThreatAd['Threat per fixture adj'] = np.zeros(len(players))
PlayerThreatAd['Threat per game adj'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerThreatAd['Threat GW'+str(j) + 'adj'] = [[] for _ in range(len(bigTable))]
    for i in range(len(bigTable)):
        for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):
            
            if toint(Fixtures[Fixtures['id']==Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==\
                PlayerThreatAd.at[i,'Team number']:
                PlayerThreatAd.at[i,'Threat GW'+str(j) + 'adj'].append(PlayerThreat.at[i,'Threat GW'+str(j)][k]\
                    *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                    Player_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
            else:
                  PlayerThreatAd.at[i,'Threat GW'+str(j) + 'adj'].append(PlayerThreat.at[i,'Threat GW'+str(j)][k]
                    *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                    Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])
            
            PlayerThreatAd.at[i,'Threat per game adj'] = PlayerThreatAd.at[i,'Threat per game adj'] +\
                PlayerThreatAd.at[i,'Threat GW'+str(j) + 'adj'][k]
    

PlayerThreatAd['Threat per fixture adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Team games'])
PlayerThreatAd['Threat per game adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Played'])
print(11)

#4 PLayers Creativity Adjusted

PlayerCreativityAd = pd.DataFrame()
PlayerCreativityAd['id'] = bigTable['id']
PlayerCreativityAd['Name'] = bigTable['full_name']
PlayerCreativityAd['Team number'] = PlayerMatches['Team number']
PlayerCreativityAd['Team'] = PlayerMatches['Team']
PlayerCreativityAd['Creativity per fixture adj'] = np.zeros(len(players))
PlayerCreativityAd['Creativity per game adj'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerCreativityAd['Creativity GW'+str(j) + 'adj'] = [[] for _ in range(len(bigTable))]
    for i in range(len(bigTable)):
        for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):
            
            if toint(Fixtures[Fixtures['id']==Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==\
                PlayerCreativityAd.at[i,'Team number']:
                PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'].append(PlayerCreativity.at[i,'Creativity GW'+str(j)][k]\
                    *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                    Player_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
            else:
                PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'].append(PlayerCreativity.at[i,'Creativity GW'+str(j)][k]\
                    *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                    Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])
            
            PlayerCreativityAd.at[i,'Creativity per game adj'] = PlayerCreativityAd.at[i,'Creativity per game adj'] +\
                PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'][k]
    

PlayerCreativityAd['Creativity per fixture adj'] = PlayerCreativityAd['Creativity per game adj']/\
noZ(PlayerMatches['Team games'])
PlayerCreativityAd['Creativity per game adj'] = PlayerCreativityAd['Creativity per game adj']/noZ(PlayerMatches['Played'])
print(12)


#Tables2Files

del TeamThreat['id']
TeamThreat.sort_values('Threat av', ascending = False, inplace = True)
TeamThreat.index = np.arange(1, len(TeamThreat) + 1)
TeamThreat.to_csv(Path('out/TeamThreat.csv'))

del TeamCreativity['id']
TeamCreativity.sort_values('Creativity av', ascending = False, inplace = True)
TeamCreativity.index = np.arange(1, len(TeamCreativity) + 1)
TeamCreativity.to_csv(Path('out/TeamCreativity.csv'))

del TableDefence['id']
TableDefence.sort_values('Threat allowed av', ascending = True, inplace = True)
TableDefence.index = np.arange(1, len(TableDefence) + 1)
TableDefence.to_csv(Path('out/TableDefence.csv'))

del TeamThreatAd['id']
TeamThreatAd.sort_values('Threat av adj', ascending = False, inplace = True)
TeamThreatAd.index = np.arange(1, len(TeamThreatAd) + 1)
TeamThreatAd.to_csv(Path('out/TeamThreatAd.csv'))

del TeamCreativityAd['id']
TeamCreativityAd.sort_values('Creativity av adj', ascending = False, inplace = True)
TeamCreativityAd.index = np.arange(1, len(TeamCreativityAd) + 1)
TeamCreativityAd.to_csv(Path('out/TeamCreativityAd.csv'))

del TableDefenceAd['id']
TableDefenceAd.sort_values('Threat allowed av adj', ascending = True, inplace = True)
TableDefenceAd.index = np.arange(1, len(TableDefenceAd) + 1)
TableDefenceAd.to_csv(Path('out/TableDefenceAd.csv'))

del TableTeams['id']
TableTeams.sort_values('Threat adjusted', ascending = False, inplace = True)
TableTeams.index = np.arange(1, len(TableTeams) + 1)
TableTeams.to_csv(Path('out/TableTeams.csv'))

del PlayerThreat['id']
PlayerThreat.sort_values('Threat per fixture', ascending = False, inplace = True)
PlayerThreat.index = np.arange(1, len(players) + 1)
PlayerThreat.to_csv(Path('out/PlayerThreat.csv'))

del PlayerCreativity['id']
PlayerCreativity.sort_values('Creativity per fixture', ascending = False, inplace = True)
PlayerCreativity.index = np.arange(1, len(players) + 1)
PlayerCreativity.to_csv(Path('out/PlayerCreativity.csv'))

del PlayerThreatAd['id']
del PlayerThreatAd['Team number']
PlayerThreatAd.sort_values('Threat per fixture adj', ascending = False, inplace = True)
PlayerThreatAd.index = np.arange(1, len(players) + 1)
PlayerThreatAd.to_csv(Path('out/PlayerThreatAd.csv'))

del PlayerCreativityAd['id']
del PlayerCreativityAd['Team number']
PlayerCreativityAd.sort_values('Creativity per fixture adj', ascending = False, inplace = True)
PlayerCreativityAd.index = np.arange(1, len(players) + 1)
PlayerCreativityAd.to_csv(Path('out/PlayerCreativityAd.csv'))

PlayerThreatAd

0
1
2
3
4
5
6
7
9
10
11
12


,Name,Team,Threat per fixture adj,Threat per game adj,Threat GW21adj,Threat GW20adj,Threat GW19adj,Threat GW18adj,Threat GW17adj,Threat GW16adj,...,Threat GW10adj,Threat GW9adj,Threat GW8adj,Threat GW7adj,Threat GW6adj,Threat GW5adj,Threat GW4adj,Threat GW3adj,Threat GW2adj,Threat GW1adj
1,Raheem Sterling,Man City,55.658630,58.441561,[0.0],[23.534534883720927],[67.31607538802659],[84.85915885416665],[44.41586488812391],[86.11890413337963],...,[55.3846349849284],[33.7060824742268],[54.97479490022171],[87.71196155099379],[0.0],[70.52714577656674],[9.725281438769906],[55.091310720268],[115.23135866666664],[68.54007871133076]
2,Raúl Jiménez,Wolves,48.321315,48.321315,[24.311149683210132],[34.125273984151065],[161.53651778242673],[48.99780653950952],[33.05817666666666],[11.670337726523886],...,[10.25279837452683],[20.813692514052118],[44.459592050209196],[12.155574841605066],[25.2795618556701],[16.997164082687334],[62.321656891495586],[44.10765783376773],[27.065941299062168],[36.89528645833333]
3,Mohamed Salah,Liverpool,47.661886,56.072807,[0.0],[34.77997228381374],[30.746072048611108],[],[116.88052732312563],[77.12783500837519],...,[87.84029799999998],[0.0],[115.60523090277776],[88.90824289405683],[47.06906976744185],[57.57340625695834],[59.494050101361125],[58.35025387263338],[26.24335143076136],[28.95324931880108]
4,Roberto Firmino,Liverpool,45.872488,45.872488,[0.0],[26.926430155210635],[77.48010156249998],[],[35.53168030623019],[48.3108417085427],...,[106.73068466666665],[54.131882598124335],[50.42355815972221],[13.074741602067181],[75.83350129198965],[22.871627143175232],[73.8546828844483],[86.2190318416523],[28.958180889115987],[48.99780653950952]
5,Marcus Rashford,Man Utd,42.399151,42.399151,[26.12697934595525],[70.77740443092962],[28.39236472945891],[29.921414994720163],[46.16419028999674],[38.53164644351464],...,[60.13367166212532],[40.95032878098128],[1.5773535960810505],[33.094173838209976],[5.557303679297089],[24.596857638888885],[106.78329202861521],[27.15212199312714],[10.09741130820399],[75.83350129198965]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,Vicente Guaita,Crystal Palace,0.000000,0.000000,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
584,Simon Moore,Sheffield Utd,0.000000,0.000000,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
585,Michael Verrips,Sheffield Utd,0.000000,0.000000,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
586,Tariq Lamptey,Chelsea,0.000000,0.000000,[0.0],[0.0],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]


In [32]:
PlayerCreativityAd = pd.DataFrame()
PlayerCreativityAd['id'] = bigTable['id']
PlayerCreativityAd['Name'] = bigTable['full_name']
PlayerCreativityAd['Team number'] = PlayerMatches['Team number']
PlayerCreativityAd['Team'] = PlayerMatches['Team']
PlayerCreativityAd['Creativity per fixture adj'] = np.zeros(len(players))
PlayerCreativityAd['Creativity per game adj'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerCreativityAd['Creativity GW'+str(j) + 'adj'] = [[] for _ in range(len(bigTable))]
    for i in range(len(bigTable)):
        for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):
            
            if toint(Fixtures[Fixtures['id']==Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==\
                PlayerCreativityAd.at[i,'Team number']:
                PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'].append(PlayerCreativity.at[i,'Creativity GW'+str(j)][k]\
                    *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                    Player_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
            else:
                PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'].append(PlayerCreativity.at[i,'Creativity GW'+str(j)][k]\
                    *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                    Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])
            
            PlayerCreativityAd.at[i,'Creativity per game adj'] = PlayerCreativityAd.at[i,'Creativity per game adj'] +\
                PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'][k]
    

PlayerCreativityAd['Creativity per fixture adj'] = PlayerCreativityAd['Creativity per game adj']/\
noZ(PlayerMatches['Team games'])
PlayerCreativityAd['Creativity per game adj'] = PlayerCreativityAd['Creativity per game adj']/noZ(PlayerMatches['Played'])

PlayerCreativityAd.sort_values('Creativity per fixture adj', ascending=False)

,id,Name,Team number,Team,Creativity per fixture adj,Creativity per game adj,Creativity GW21adj,Creativity GW20adj,Creativity GW19adj,Creativity GW18adj,...,Creativity GW10adj,Creativity GW9adj,Creativity GW8adj,Creativity GW7adj,Creativity GW6adj,Creativity GW5adj,Creativity GW4adj,Creativity GW3adj,Creativity GW2adj,Creativity GW1adj
303,215,Kevin De Bruyne,11,Man City,52.235625,54.847406,[31.50705987292277],[23.27304005167958],[15.482697339246117],[106.50439357638886],...,[60.789641531650325],[32.582546391752565],[0.0],[38.77791984359726],[102.29383751319958],[27.31998910081743],[25.966501441515646],[35.85172989949748],[120.61511886666663],[34.64052626761852]
390,283,Emiliano Buendía,14,Norwich,43.845313,43.845313,[58.23662027491408],[29.657907066666656],[48.24468806518462],[128.01273669623055],...,[31.494913511635975],[48.14132998324957],[25.89064864355689],[42.88162714776631],[15.694120112945262],[59.427654707112964],[4.908951583379094],[54.652419896640815],[49.528902916944986],[69.10367981790591]
262,182,Trent Alexander-Arnold,10,Liverpool,42.097910,42.097910,[0.0],[41.62377328159644],[68.01031137152776],[],...,[94.73528913333331],[40.84496596040292],[84.9821431423611],[10.067551033591728],[38.9627299741602],[80.91823947895789],[34.15779083405733],[69.93321471600687],[26.695823007153802],[26.948793596730237]
573,415,João Filipe Iria Santos Moutinho,20,Wolves,37.444885,37.444885,[34.22261840021119],[35.660911313437865],[112.92736380753135],[22.271730245231602],...,[24.764451458472493],[4.162738502810424],[23.71178242677824],[6.825822795670537],[16.197645189003435],[75.83350129198965],[1.0386942815249265],[37.132493339125396],[47.73447829107327],[36.64931788194444]
246,171,James Maddison,9,Leicester,36.765992,38.604292,[36.515735749276324],[22.970188541094632],[2.9006482886528406],[17.93203546025104],...,[18.46084031681144],[33.43975919490298],[24.399570898668014],[0.0],[29.185647399999993],[11.441511549149007],[16.696904941373532],[48.37654392764857],[68.64239341085269],[55.199181818181806]
206,141,Lucas Digne,8,Everton,33.066645,34.719977,[21.93339874476987],[0.0],[28.208385823921226],[14.195658777969017],...,[41.3324461147721],[45.940377082189265],[27.490354184766865],[39.124441004184085],[25.23425129198966],[34.58039195979898],[40.950612527716174],[35.63300612283345],[27.864317713833152],[15.354993127147763]
103,49,Pascal Groß,4,Brighton,31.093894,36.276209,[0.0],[0.0],[43.73124513333332],[6.668118217054261],...,[2.885261893124796],[46.50974769216276],[54.782121333333315],[43.80038436692506],[32.57235175907369],[70.87998037938023],[1.1855891213389118],[13.483652976494632],[67.33599624748305],[10.191981982576555]
216,151,Gylfi Sigurdsson,8,Everton,30.255297,31.768061,[60.168647907949776],[26.736143453573806],[71.08513227628148],[1.4805288296041306],...,[1.3615394014277866],[0.7409738239062784],[28.92641746307558],[99.73768483263596],[75.83350129198965],[12.459111809045224],[80.33051662971172],[15.347549453654858],[24.12414083949313],[12.26526890034364]
39,29,Jack Grealish,2,Aston Villa,30.056485,33.220325,[34.77324652476107],[33.47458302534318],[25.09281607629427],[77.1916509325498],...,[3.7049660041841],[67.29894755628774],[20.415752724795635],[37.029917390674775],[40.32263812392426],[31.213522332051983],[3.557864261168384],[15.580414222873896],[59.329103852596305],[20.873877266666664]
156,113,Willian Borges Da Silva,6,Chelsea,27.128636,28.485068,[16.921989703459634],[79.94855679862304],[29.772629726622373],[66.39970913333332],...,[36.209309803069786],[35.332720552215534],[37.82662378640776],[13.809899643053265],[16.380131512392513],[7.1803813747228356],[3.6609276485788103],[0.22271730245231602],[1.2298428819444442],[0.0]


In [8]:
#169.24
threatAllowedAv

168.66416666666663

In [23]:
PlayerThreatAd.sort_values('Threat per fixture adj', ascending=False)

,id,Name,Team number,Team,Threat per fixture adj,Threat per game adj,Threat GW21adj,Threat GW20adj,Threat GW19adj,Threat GW18adj,...,Threat GW10adj,Threat GW9adj,Threat GW8adj,Threat GW7adj,Threat GW6adj,Threat GW5adj,Threat GW4adj,Threat GW3adj,Threat GW2adj,Threat GW1adj
302,214,Raheem Sterling,11,Man City,71.729404,75.315874,[0.0],[23.534534883720927],[88.91918410041839],[84.85915885416665],...,[55.3846349849284],[53.351510460251035],[54.97479490022171],[112.63096652719662],[0.0],[140.78870815899577],[9.725281438769906],[96.3291161087866],[115.23135866666664],[109.66699372384934]
267,187,Roberto Firmino,10,Liverpool,60.693623,60.693623,[0.0],[26.926430155210635],[107.49461305007584],[],...,[106.73068466666665],[75.07560276513234],[50.42355815972221],[17.062636992075532],[98.96329455403809],[22.871627143175232],[122.85098634294384],[86.2190318416523],[54.600438374641705],[48.99780653950952]
271,191,Mohamed Salah,10,Liverpool,59.191816,69.637430,[0.0],[34.77997228381374],[42.656592480188834],[],...,[87.84029799999998],[0.0],[115.60523090277776],[116.02593154611361],[61.42549317147192],[57.57340625695834],[98.96329455403809],[58.35025387263338],[49.48164727701904],[28.95324931880108]
299,211,Gabriel Fernando de Jesus,11,Man City,52.727682,65.134196,[107.3317424242424],[0.0],[0.0],[125.4439739583333],...,[32.02966842501883],[189.6942594142259],[11.219345898004432],[87.43719769874475],[0.0],[72.61733368200835],[0.0],[0.0],[45.336927999999986],[62.243428870292874]
567,409,Raúl Jiménez,20,Wolves,51.491560,51.491560,[29.170299334811524],[22.438691796008865],[161.53651778242673],[74.04768292682924],...,[14.585149667405762],[20.813692514052118],[33.658037694013295],[12.155574841605066],[30.292233924611963],[16.997164082687334],[60.58446784922393],[44.10765783376773],[27.065941299062168],[33.658037694013295]
328,233,Marcus Rashford,12,Man Utd,47.890901,47.890901,[36.908101771448415],[84.88863407433135],[28.39236472945891],[39.36864188954497],...,[99.65187478291071],[40.95032878098128],[2.4605401180965605],[33.094173838209976],[7.381620354289682],[24.596857638888885],[145.17186696769707],[27.15212199312714],[11.072430531434524],[75.83350129198965]
167,460,Tammy Abraham,6,Chelsea,47.192465,49.552088,[49.684018087855286],[112.44277777777775],[12.669204138988244],[44.45412144702841],...,[6.537370801033591],[49.68663827655309],[45.76159560723514],[74.88466707852827],[42.656592480188834],[74.52602713178294],[79.7559237726098],[62.75875968992247],[6.149214409722221],[31.379379844961235]
272,192,Sadio Mané,10,Liverpool,43.085801,45.353475,[0.0],[45.99931818181817],[75.07560276513234],[],...,[20.779425333333325],[6.825054796830213],[55.34292968749999],[54.600438374641705],[10.23758219524532],[52.05266867067467],[63.131756870679475],[26.12697934595525],[64.83802056988702],[4.45434604904632]
241,166,Jamie Vardy,9,Leicester,41.682950,46.070629,[0.0],[0.0],[10.23758219524532],[38.12512934027777],...,[114.37538802083331],[21.540949174630754],[12.298428819444442],[50.47531507459362],[42.50336999999998],[4.919371527777777],[38.140138190954765],[33.205757812499996],[12.298428819444442],[4.4877383592017726]
115,502,Neal Maupay,4,Brighton,39.947666,39.947666,[45.76159560723514],[21.18896566164154],[0.0],[18.30463824289405],...,[33.469037960247626],[57.37916048874244],[69.89443066666665],[1.945056287753981],[92.39017366831409],[38.9788604112366],[9.725281438769906],[84.15971320899334],[12.967041918359872],[18.47803473366282]


In [7]:
TeamCreativity

,id,Team,Creativity av,Matches,Creativity GW21,Creativity GW20,Creativity GW19,Creativity GW18,Creativity GW17,Creativity GW16,...,Creativity GW10,Creativity GW9,Creativity GW8,Creativity GW7,Creativity GW6,Creativity GW5,Creativity GW4,Creativity GW3,Creativity GW2,Creativity GW1
0,1,Arsenal,147.780952,21,[132.1],[98.5],[193.79999999999998],[87.59999999999998],[69.19999999999997],[123.19999999999999],...,[173.7],[163.7],[157.00000000000003],[128.7],[226.90000000000003],[100.9],[263.1],[92.5],[209.70000000000002],[128.5]
1,2,Aston Villa,148.552381,21,[119.79999999999998],[149.7],[160.29999999999998],[220.99999999999994],[113.6],[202.99999999999997],...,[83.60000000000001],[238.50000000000003],[192.3],[181.2],[148.89999999999995],[125.4],[107.30000000000001],[96.80000000000001],[258.5],[89.9]
2,3,Bournemouth,109.661905,21,[76.7],[98.2],[108.60000000000001],[59.60000000000001],[105.8],[43.199999999999996],...,[148.9],[111.69999999999999],[114.00000000000001],[174.5],[57.9],[118.09999999999998],[119.5],[79.3],[137.7],[113.60000000000001]
3,4,Brighton,152.790476,21,[160.1],[197.8],[111.7],[136.29999999999998],[166.5],[123.2],...,[86.39999999999999],[228.4],[180.8],[115.1],[216.6],[155.10000000000002],[73.0],[163.10000000000002],[204.79999999999998],[75.7]
4,5,Burnley,115.419048,21,[243.9],[102.1],[81.0],[30.5],[93.9],[88.5],...,[146.3],[124.6],[116.9],[121.8],[184.9],[82.9],[92.4],[131.7],[182.60000000000002],[105.2]
5,6,Chelsea,189.619048,21,[157.7],[146.1],[167.7],[166.0],[194.20000000000005],[205.4],...,[185.8],[205.20000000000002],[151.4],[228.1],[175.6],[161.6],[151.9],[270.1],[176.59999999999997],[207.6]
6,7,Crystal Palace,107.985714,21,[145.79999999999998],[60.5],[140.89999999999998],[180.39999999999998],[124.49999999999999],[38.2],...,[124.4],[74.8],[96.1],[182.5],[110.1],[120.19999999999999],[229.50000000000003],[30.000000000000004],[79.10000000000001],[69.7]
7,8,Everton,149.157143,21,[95.00000000000001],[225.3],[252.7],[81.39999999999999],[58.1],[129.1],...,[111.5],[176.2],[129.3],[154.79999999999998],[184.3],[140.70000000000002],[188.9],[152.3],[119.0],[114.4]
8,9,Leicester,163.633333,21,[190.40000000000003],[140.2],[44.2],[76.6],[206.8],[278.5],...,[258.29999999999995],[226.6],[54.1],[198.20000000000002],[145.0],[82.0],[185.29999999999998],[138.40000000000003],[124.3],[175.50000000000003]
9,10,Liverpool,177.385000,20,[0.0],[147.5],[219.70000000000005],[],[116.3],[266.2],...,[255.10000000000002],[156.0],[207.8],[121.30000000000001],[101.69999999999999],[314.1],[167.1],[283.29999999999995],[169.1],[179.3]


In [43]:
TeamThreat = pd.DataFrame()
TeamThreat['id'] = pd.DataFrame(d1['teams'])['id']
TeamThreat['Team'] = pd.DataFrame(d1['teams'])['name']
TeamThreat['Threat av'] = np.zeros(len(TeamThreat))
TeamThreat['Matches'] = TeamMatches['Matches']


for j in range(lastGW,0,-1):
    TeamThreat['Threat GW'+str(j)] = np.zeros(len(TeamThreat))
    GWfixturesPD = [Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['fixture'] \
                                      for i in range(1,team_number+1)]
    GWfixtures = [list(set(pd2int(a))) for a in GWfixturesPD]
    for i in range(1,team_number+1):
        print(j, i)
        for k in range(len(GWfixtures[i-1])):
            TeamThreat.at[i-1,'Threat GW'+str(j)] = Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['threat'].sum()
TeamThreat

19 1
19 2
19 3
19 4
19 5
19 6
19 7
19 8
19 9
19 10
19 11
19 12
19 13
19 14
19 15
19 16
19 17
19 18
19 19
19 20


TypeError: 'int' object is not iterable

In [6]:
nexTour

""


In [7]:
Fixtures[(Fixtures['finished'])&(Fixtures['event']==19)&((Fixtures['team_a']==20) | (Fixtures['team_h']==20))]['id']

188    190
Name: id, dtype: int64

In [49]:
def pd2int(a):
    if len(a)==0:
        return 0
    elif len(a)==1:
        return int(a)
    else: return [int(i) for i in a]

def pdseries2int(A):
    return [pd2int(set(pd2int(i))) for i in A]
    
pd2int(Fixtures[(Fixtures['event']==24)&((Fixtures['team_a']==1) | (Fixtures['team_h']==20))]['id'])

A = [Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==18)]['fixture'] \
                                      for i in range(1,team_number+1)]
B = [list(set(pd2int(a))) for a in A]
B

TypeError: 'int' object is not iterable

In [38]:
b = {}
a = [Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==19)]['fixture'] \
                                      for i in range(1,team_number+1)]
TeamThreat['Threat GW19'] = [Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==19)]['threat'].sum() \
                                      for i in range(1,team_number+1)]
a

[9932     182
 9975     182
 10000    182
 10001    182
 10002    182
 10003    182
 10004    182
 10005    182
 10006    182
 10007    182
 10008    182
 10009    182
 10040    182
 10164    182
 10178    182
 10196    182
 10197    182
 10199    182
 10200    182
 10201    182
 10202    182
 10387    182
 10394    182
 10395    182
 10396    182
 10418    182
 10431    182
 10433    182
 10434    182
 10435    182
 10498    182
 Name: fixture, dtype: object, 9933     181
 9940     181
 9976     181
 9981     181
 10010    181
 10011    181
 10012    181
 10013    181
 10113    181
 10156    181
 10159    181
 10161    181
 10162    181
 10203    181
 10204    181
 10205    181
 10206    181
 10207    181
 10208    181
 10209    181
 10210    181
 10211    181
 10386    181
 10388    181
 10397    181
 10436    181
 10484    181
 Name: fixture, dtype: object, 9936     182
 9937     182
 9938     182
 9983     182
 10021    182
 10022    182
 10023    182
 10024    182
 10025    182
 1

In [23]:
B  = pd.DataFrame()
len(B)

0

In [36]:
a={1}
list(a)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [32]:
type(a)

set

In [65]:
url = "https://fantasy.premierleague.com/api/event/18/live/"
p = requests.get(url)

d = json.loads(p.text)
nexTour = pd.DataFrame(d['elements'])
nt1 = pd.DataFrame(nexTour['stats'].tolist())
nt1['id'] = nexTour['id']
nt1['gameweek'] = i
nt1['fixture'] = [nexTour.at[i,'explain'] for i in nexTour.index]
nt1.to_csv(Path('in/nt1.csv'))
nt1



,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,...,bps,influence,creativity,threat,ict_index,total_points,in_dreamteam,id,gameweek,fixture
0,90,0,0,1,0,0,0,0,0,0,...,21,5.2,0.0,0.0,0.5,6,False,14,18,"[{'fixture': 174, 'stats': [{'identifier': 'mi..."
1,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,False,27,18,"[{'fixture': 171, 'stats': [{'identifier': 'mi..."
2,90,0,0,0,1,0,0,0,0,0,...,14,14.2,0.0,0.0,1.4,2,False,47,18,"[{'fixture': 173, 'stats': [{'identifier': 'mi..."
3,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,False,48,18,"[{'fixture': 173, 'stats': [{'identifier': 'mi..."
4,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,False,72,18,"[{'fixture': 172, 'stats': [{'identifier': 'mi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,13,0,0,0,0,0,0,0,0,0,...,3,0.0,0.0,0.0,0.0,1,False,554,18,"[{'fixture': 176, 'stats': [{'identifier': 'mi..."
574,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,False,559,18,"[{'fixture': 177, 'stats': [{'identifier': 'mi..."
575,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,False,561,18,"[{'fixture': 174, 'stats': [{'identifier': 'mi..."
576,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,False,570,18,"[{'fixture': 174, 'stats': [{'identifier': 'mi..."


In [46]:
Gameweeks.columns[21]

'team'